# Tarea 07 - Chatbot RAG

***Inteligencia Artificial***

***II Semestre 2024***

***Tecnológico de Costa Rica***

Estudiantes: 
- Esteban Guzmán
- Rolando Mora

In [25]:
import os
import re
import time
from pinecone.grpc import PineconeGRPC
from pinecone import ServerlessSpec

INDEX_NAME = "chatbotrag"
NAMESPACE = f"{INDEX_NAME}-namespace"

## Parte 1: Preparación del Corpus y Cálculo de Embeddings

### 1. Elección del Dataset

Se escogió el dataset de `BBC Full Text Document Classification` del sitio web de [Kaggle](https://www.kaggle.com/datasets/shivamkushwaha/bbc-full-text-document-classification).

Se usarán los datos referentes a tecnología.

In [26]:
# Extraemos el contenido de los artículos
def extract_articles(folder_path: str, max_articles) -> list[str]:
	articles = []
	for filename in os.listdir(folder_path)[:max_articles]:
		with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as file:
			content = file.read()
			articles.append(content)
	return articles

### 2. Preprocesamiento

In [27]:
# Limpiar el texto
def clean_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'[^a-zA-Záéíóúñü\s.]', '', text)  # Eliminar caracteres especiales, manteniendo puntos
    text = re.sub(r'\s+', ' ', text).strip()  # Eliminar espacios extra
    return text

# Segmentar el texto en párrafos de 4 oraciones
def segment_text(text, doable=True):
    if not doable: return [text]
    sentences_per_fragment = 4
    # Dividir el texto en oraciones
    sentences = text.split(".")
    start = 0
    fragments = []
    # Separa el texto en la cantidad de oraciones
    while start < len(sentences):
        end = start + sentences_per_fragment
        fragments.append(".".join(sentences[start:end]))
        start = end
    return fragments

### 3. Generación de Embeddings

In [28]:
def index_articles(articles):
	plain_articles = []
	for i, article in enumerate(articles):
		for j, paragraph in enumerate(article):
			plain_articles.append({"id": f"{i},{j}", "text": paragraph})
	return plain_articles

def generate_embeddings(pc, indexed_articles):
	embeddings = pc.inference.embed(
		model="multilingual-e5-large",
		inputs=[a["text"] for a in indexed_articles],
		parameters={"input_type": "passage", "truncate": "END"}
	)

	return embeddings

## Parte 2: Configuración de la Base de Datos Vectorial

### 1. Elección de la base de datos vectorial
Se seleccionó Pinecone debido a su almacenamiento en la nube y al no ser necesario más instalaciones más que la librería de python.

In [29]:
def get_pinecone():
    return PineconeGRPC(api_key="81002466-39d5-4599-a684-9b08a1c70aee")

def delete_index(pc):
    pc.delete_index(INDEX_NAME)

### 2. Carga de datos

In [30]:
def save_embeddings(pc, indexed_articles, embeddings):
	# Creación del índice de pinecone
	if not pc.has_index(INDEX_NAME):
		pc.create_index(
			name=INDEX_NAME,
			dimension=1024,
			metric="cosine",
			spec=ServerlessSpec(
				cloud="aws", 
				region="us-east-1"
			) 
		) 

	# Se espera hasta que el índice esté listo
	while not pc.describe_index(INDEX_NAME).status["ready"]:
		time.sleep(1)

	index = pc.Index(INDEX_NAME)

	# Se preparan los registros
	records = []
	for d, e in zip(indexed_articles, embeddings):
		records.append({
			"id": d["id"],
			"values": e["values"],
			"metadata": {"text": d["text"]}
		})

	index.upsert(
		vectors=records,
		namespace=NAMESPACE
	)

	# Se espera la indexación de los vectores
	time.sleep(10)

	return index

## Parte 3: Diseño de Chatbot con RAG

### 1. Desarrollo del flujo de recuperación

In [31]:
def answer_query(pc, index, query):
	# Se hace embedding al query
	query_embedding = pc.inference.embed(
		model="multilingual-e5-large",
		inputs=[query],
		parameters={
			"input_type": "query"
		}
	)

	# Se realiza la consulta a la base de datos
	results = index.query(
		namespace=NAMESPACE,
		vector=query_embedding[0].values,
		top_k=3,
		include_values=False,
		include_metadata=True
	)

	return results

### 2. Generación de respuestas

In [32]:
def contextualize_answer(query, answer):
	return answer["matches"][0]["metadata"]["text"].replace(".", ".\n")


### 3. Afinación del flujo de interacción

In [ ]:
# Interacción manual, aparece un input (una caja flotante) que preguntará por la consulta
try:
	# Se limita a los primeros 95 artículos por limitaciones de la base de datos
	articles = extract_articles("./dataset/bbc-fulltext (document classification)/bbc/tech", 95)
	articles = list(map(clean_text, articles))

	# No se segmentan los artículos para no pasar la limitación en la base de datos
	articles = list(map(lambda article: segment_text(article, False), articles))

	pc = get_pinecone()
	indexed_articles = index_articles(articles)

	embeddings = generate_embeddings(pc, indexed_articles)
	index = save_embeddings(pc, indexed_articles, embeddings)

	print("Inserta una consulta en el input!!! (Escapar para salir)")
	query = input("Consulta: ")
	while query:
		print("Consulta: ", query)
		answer = answer_query(pc, index, query)
		context = contextualize_answer(query, answer)
		print("Respuesta:", context)
		query = input("Consulta: ")
		
	delete_index(pc)
except Exception as ex:
	print("Error: ", ex)

Inserta una consulta en el input!!! (Escapar para salir)
Consulta:  Whose toolbar sparks concern?
Respuesta: googles toolbar sparks concern search engine firm google has released a trial tool which is concerning some net users because it directs people to preselected commercial websites.
 the autolink feature comes with googles latest toolbar and provides links in a webpage to amazon.
com if it finds a books isbn number on the site.
 it also links to googles map service if there is an address or to car firm carfax if there is a licence plate.
 google said the feature available only in the us adds useful links.
 but some users are concerned that googles dominant position in the search engine market place could mean it would be giving a competitive edge to firms like amazon.
 autolink works by creating a link to a website based on information contained in a webpage even if there is no link specified and whether or not the publisher of the page has given permission.
 if a user clicks the 

## Parte 4: Pruebas y Evaluación del Chatbot

### 1. Pruebas de funcionalidad

In [34]:
try:
	# Se limita a los primeros 95 artículos por limitaciones de la base de datos
	articles = extract_articles("./dataset/bbc-fulltext (document classification)/bbc/tech", 95)
	articles = list(map(clean_text, articles))

	# No se segmentan los artículos para no pasar la limitación en la base de datos
	articles = list(map(lambda article: segment_text(article, False), articles))

	pc = get_pinecone()
	indexed_articles = index_articles(articles)

	embeddings = generate_embeddings(pc, indexed_articles)
	index = save_embeddings(pc, indexed_articles, embeddings)

	queries = [
		"What helps drive democracy in Asia?",
		"What happened to China net cafe culture?",
		"What is Microsoft seeking?",
		"What gets the creative bug?",
		"Who is Nicholas Negroponte?"
	]
	
	for query in queries:
		print("Consulta: ", query)
		answer = answer_query(pc, index, query)
		context = contextualize_answer(query, answer)
		print("Respuesta:", context)
		
	delete_index(pc)
except Exception as ex:
	print("Error: ", ex)

Consulta:  What helps drive democracy in Asia?
Respuesta: ink helps drive democracy in asia the kyrgyz republic a small mountainous state of the former soviet republic is using invisible ink and ultraviolet readers in the countrys elections as part of a drive to prevent multiple voting.
 this new technology is causing both worries and guarded optimism among different sectors of the population.
 in an effort to live up to its reputation in the s as an island of democracy the kyrgyz president askar akaev pushed through the law requiring the use of ink during the upcoming parliamentary and presidential elections.
 the us government agreed to fund all expenses associated with this decision.
 the kyrgyz republic is seen by many experts as backsliding from the high point it reached in the mids with a hastily pushed through referendum in reducing the legislative branch to one chamber with deputies.
 the use of ink is only one part of a general effort to show commitment towards more open elect

## Parte 5: Informe Final

### Introducción
RAG (Retrieval-Augmented Generation) es un enfoque de procesamiento de lenguaje natural que combina la recuperación de información y la generación de respuestas. Primero, busca fragmentos relevantes desde una base de datos usando embeddings y bases vectoriales como FAISS o Pinecone, y luego utiliza un modelo generativo (como GPT-3 o LLaMA) para crear respuestas contextuales basadas en esos fragmentos, permitiendo generar respuestas precisas y fundamentadas en un corpus específico.

**Objetivos:**
* Sea capaz de responder preguntas basadas en un corpus de documentos.
* Use el enfoque de Retrieval-Augmented Generation (RAG) para combinar recuperación de información y generación de respuestas.
* Una base de datos vectorial para almacenar y consultar embeddings del corpus.
* Un modelo generativo para generar respuestas coherentes basadas en los fragmentos recuperados.
* Provea respuestas relevantes incluso en un corpus amplio, demostrando la eficacia del enfoque RAG.

### Preparación del corpus

**Carga del Dataset:**
* Se cargaron los documentos, ya sea desdes archivos text en carpetas organizadas por categorías.
* Cada artículo fue leído y almacenado como una entrada de texto.

**Limpieza del Texto:**
* Se eliminaron caracteres especiales, números innecesarios y puntuación no esencial.
* Todo el texto fue convertido a minúsculas para garantizar la uniformidad.
* Se eliminaron espacios adicionales y se reemplazaron por un único espacio.

**Segmentación:**
* Los textos fueron divididos en fragmentos de 3 a 4 oraciones, lo que facilita el almacenamiento y recuperación de información en bases de datos vectoriales.
* Esta segmentación asegura que cada fragmento sea lo suficientemente pequeño para ser manejado por modelos generativos.

#### Generación de Embeddings
La generación de embeddings se realiza utilizando Pinecone y un modelo preentrenado llamado multilingual-e5-large, diseñado para generar representaciones vectoriales de textos en múltiples idiomas. Este proceso transforma cada párrafo en un vector numérico de alta dimensionalidad (1024 dimensiones en este caso), que captura las características semánticas del texto.

Los textos de los artículos son preprocesados y organizados en una lista plana, donde cada párrafo se identifica mediante un ID único (id) y se asocia con su contenido (text). Este preprocesamiento asegura que cada párrafo pueda ser identificado y recuperado fácilmente después de que se generen los embeddings. Se utiliza la API de Pinecone para realizar la inferencia con el modelo multilingual-e5-large.

Cada párrafo es convertido en un vector de 1024 dimensiones. Estos embeddings representan las relaciones semánticas del texto, lo que significa que textos similares tendrán vectores cercanos en el espacio vectorial. Los embeddings generados se devuelven como una lista, donde cada vector corresponde al párrafo en la misma posición de la lista de entrada.

### Diseño e implementación del flujo RAG
Se solicita al usuario la consulta, se transforma en embedding, se hace la solicitud a la base de datos a partir de los embeddings generados de los artículos extraídos previamente. Se obtiene el artículo que más similitudes presenta con la consulta hecha.

### Pruebas y resultados
```python
# Ejemplos de consultas
[
"What helps drive democracy in Asia?",
"What happened to China net cafe culture?",
"What is Microsoft seeking?",
"What gets the creative bug?",
"Who is Nicholas Negroponte?"
]
```
Funciona mejor con preguntas que puedan ser sacadas directamente de los artículos y tiene un rendimiento menor con textos diferentes.

### Conclusión
Los chatbots RAG combinan técnicas de recuperación de información y generación de texto, lo que permite generar respuestas más precisas y contextualmente relevantes. Esto es especialmente útil cuando se interactúa con datos complejos o fuentes de información vastas. Utilizar datos de la BBC organizados en carpetas por temas te permitirá estructurar el contenido de manera eficiente, lo cual es crucial para una recuperación de información más precisa. Los archivos de texto dentro de cada carpeta representan documentos que contienen información relevante que el chatbot podrá usar para generar respuestas contextuales según la consulta del usuario. Pinecone es una plataforma especializada en el manejo de vectores de alta dimensión, lo que es perfecto para trabajar con modelos de lenguaje que generan embeddings (representaciones numéricas) de texto. Al utilizar Pinecone, puedes indexar los documentos de texto de la BBC (por tema) y convertir esos archivos en vectores que puedan ser recuperados de manera eficiente mediante consultas. El uso de datos organizados por temas de la BBC junto con Pinecone para almacenar y recuperar vectores es una combinación poderosa para desarrollar un chatbot RAG eficiente. Este enfoque no solo mejora la relevancia y precisión de las respuestas generadas, sino que también permite un sistema escalable, rápido y flexible. Sin embargo, es fundamental prestar atención a la calidad de los datos, la optimización del sistema de recuperación y la mejora continua para asegurar que el chatbot proporcione respuestas confiables y contextualmente apropiadas.

### 2. Evaluación de calidad

Debido a limitaciones en la cantidad de inputs que puede recibir la base de datos, se usaron los primeros 95 artículos.

El chatbot funciona mejor con preguntas que puedan ser sacadas directamente de los artículos.
Sin embargo, consultas que puedan no responderse directamente del texto debido al cambio en las palabras, presentarían problemas. 